# Imports

Due to a limitation of Pyodide (see also [the Limitations page of the documentation](https://docs.brightway.dev/projects/live/content/limitations.html#)), a patch is required to use Brightway in the JupyterLite environment:

In [ ]:
import micropip
await micropip.install(
    'https://files.brightway.dev/fs-2.5.1-py2.py3-none-any.whl'
)

Due to a limitation of Web Assembly, SQL databases cannot be stored in every directory of the file system. The `BRIGHTWAY_DIR` environment variable must be set to make sure the `/tmp/` directory is used for databases.

In [ ]:
import os
os.environ["BRIGHTWAY_DIR"] = "/tmp/"

Now we can import the Brightway packages. The Pyodide kernel of the JupyterLite environment uses a [the `micropip` package](https://micropip.pyodide.org/en/stable/) to install other packages. Make sure not to change the exact versions of the Brightway packages. Only the versions listed below have been optimized for use in the JupyterLite environment.

In [ ]:
await micropip.install('bw2data==4.0.dev29', keep_going = True)
await micropip.install('bw2io==0.9.dev23', keep_going = True)
await micropip.install('bw2calc==2.0.dev14', keep_going = True)

In [ ]:
import bw2data
import bw2calc
import bw2io

# Brightway Functionality

Due to a limitation of Pyodide (see also [the Limitations page of the documentation](https://docs.brightway.dev/projects/live/content/limitations.html#)), Brightway project backup files cannot be loaded at the moment. But we can create a sample database:

In [ ]:
bw2io.add_example_database(searchable=False)

We can now do normal Brightway operations in the Browser:

In [ ]:
bw2data.databases

In [ ]:
for obj in bw2data.Database("Mobility example"):
    print(obj)

In [ ]:
co2 = bw2data.get_node(name="CO2")

In [ ]:
gwp = bw2data.Method(("GWP", "simple"))
gwp.write([(co2.key, 1)])

In [ ]:
lca = bw2calc.LCA({bw2data.get_node(name="Electric car"):1}, method=("GWP", "simple"))

In [ ]:
lca.lci()
lca.lcia()
lca.score